In [53]:
import duckdb
import os
import awswrangler as wr
from pathlib import Path

import dotenv

_root = Path.cwd().parent.parent
dotenv.load_dotenv(_root / ".env.airflow", override=True)

con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL aws; LOAD aws;")
con.execute(f"SET s3_access_key_id='{os.environ['AWS_ACCESS_KEY_ID']}'")
con.execute(f"SET s3_secret_access_key='{os.environ['AWS_SECRET_ACCESS_KEY']}'")
con.execute(f"SET s3_region = 'eu-west-1'")

con.sql("""
    SELECT * FROM read_parquet('s3://coding-projects-taradim-2026/silver/*.parquet')
    LIMIT 10
""").show()

┌──────────────┬─────────────┬───────────────────┬───────────┬──────────────┬─────────────┬────────────────┬─────────────────┬────────────────┬────────────────────┬───────────────────┬────────────────┬───────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬───────────────────────┬───────────────────┬────────────────┬──────────────────────────┬──────────────────┬────────────────┬──────────────────────────┬──────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────┬────────────────┐
│   coin_id    │ coin_symbol │     coin_name     │ price_usd │ high_24h_usd │ low_24h_usd │ market_cap_usd │ market_cap_rank │ volume_24h_usd │ circulating_supply │   total_supply    │   max_supply   │  price_change_pct_7d  │ price_change_pct_14d  │ price_change_pct_30d │ price_change_pct_200d │  price_change_pct_1y  │ all_time_high_usd │ ath_change_pct │       ath_date_at        │ all_time_low_usd │ atl_change

In [63]:
import awswrangler as wr

# Lister les bases Glue
dbs = wr.catalog.get_databases()
for db in dbs:
    print(db["Name"])

# Lister les tables d'une base
tables = wr.catalog.get_tables(database="data_pipeline_portfolio")
for t in tables:
    print(t["Name"])

data_pipeline_portfolio
gold_crypto_prices
silver_coingecko
silver_defillama
silver_yfinance


In [62]:
import boto3

client = boto3.client("glue", region_name="eu-west-1")  # ta région

# Lister les bases
for db in client.get_databases()["DatabaseList"]:
    print(db["Name"])

# Lister les tables d'une base
for t in client.get_tables(DatabaseName="data_pipeline_portfolio")["TableList"]:
    print(t["Name"])

data_pipeline_portfolio
gold_crypto_prices
silver_coingecko
silver_defillama
silver_yfinance


In [65]:
df = wr.s3.read_parquet_table(
    database="data_pipeline_portfolio",
    table="silver_coingecko"  # ou silver_defillama, silver_yfinance, gold_crypto_prices
)

InvalidSchemaConvergence: At least 2 different schemas were detected:
    1 - coin_id: string
coin_symbol: string
coin_name: string
price_usd: double
high_24h_usd: double
low_24h_usd: double
market_cap_usd: int64
market_cap_rank: int64
volume_24h_usd: double
circulating_supply: double
total_supply: double
max_supply: double
price_change_pct_7d: double
price_change_pct_14d: double
price_change_pct_30d: double
price_change_pct_200d: double
price_change_pct_1y: double
all_time_high_usd: double
ath_change_pct: double
ath_date_at: string
all_time_low_usd: double
atl_change_pct: double
atl_date_at: string
last_updated_at: string
coin_image_url: string
ingestion_date: date32[day]
    2 - protocol_id: string
protocol_name: string
protocol_slug: string
protocol_symbol: string
protocol_category: string
primary_chain: string
tvl_usd: double
market_cap_usd: double
tvl_change_pct_1h: double
tvl_change_pct_1d: double
tvl_change_pct_7d: double
protocol_url: string
protocol_logo_url: string
listed_at_